# Machine Learning Engineer Nanodegree
## Deep Learning
## Project: Build a Digit Recognition Program

In this notebook, a template is provided for you to implement your functionality in stages which is required to successfully complete this project. If additional code is required that cannot be included in the notebook, be sure that the Python code is successfully imported and included in your submission, if necessary. Sections that begin with **'Implementation'** in the header indicate where you should begin your implementation for your project. Note that some sections of implementation are optional, and will be marked with **'Optional'** in the header.

In addition to implementing code, there will be questions that you must answer which relate to the project and your implementation. Each section where you will answer a question is preceded by a **'Question'** header. Carefully read each question and provide thorough answers in the following text boxes that begin with **'Answer:'**. Your project submission will be evaluated based on your answers to each of the questions and the implementation you provide.

>**Note:** Code and Markdown cells can be executed using the **Shift + Enter** keyboard shortcut. In addition, Markdown cells can be edited by typically double-clicking the cell to enter edit mode.

----
## Step 1: Design and Test a Model Architecture
Design and implement a deep learning model that learns to recognize sequences of digits. Train the model using synthetic data generated by concatenating character images from [notMNIST](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html) or [MNIST](http://yann.lecun.com/exdb/mnist/). To produce a synthetic sequence of digits for testing, you can for example limit yourself to sequences up to five digits, and use five classifiers on top of your deep network. You would have to incorporate an additional ‘blank’ character to account for shorter number sequences.

There are various aspects to consider when thinking about this problem:
- Your model can be derived from a deep neural net or a convolutional network.
- You could experiment sharing or not the weights between the softmax classifiers.
- You can also use a recurrent network in your deep neural net to replace the classification layers and directly emit the sequence of digits one-at-a-time.

Here is an example of a [published baseline model on this problem](http://static.googleusercontent.com/media/research.google.com/en//pubs/archive/42241.pdf). ([video](https://www.youtube.com/watch?v=vGPI_JvLoN0))

### Implementation
Use the code cell (or multiple code cells, if necessary) to implement the first step of your project. Once you have completed your implementation and are satisfied with the results, be sure to thoroughly answer the questions that follow.

In [4]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

from tensorflow.examples.tutorials.mnist import input_data
import tensorflow as tf

import numpy as np
import matplotlib.pyplot as plt

In [5]:
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

In [6]:
print("There are {} training examples".format(mnist.train.num_examples))
print("There are {} validation examples".format(mnist.validation.num_examples))
print("There are {} testing examples".format(mnist.test.num_examples))

In [7]:
key = '0123456789'
fig, axes = plt.subplots(1, 10, figsize=(10, 2))
for ax, image, label in zip(axes, mnist.train.images, mnist.train.labels):
    ax.set_axis_off()
    ax.set_title(key[np.argmax(label)])
    ax.imshow(image.reshape((28, 28)))
display()

In [8]:
five_numbers = [0, 3, 4, 5, 7]

idx_03457 = np.any(mnist.train.labels[:, five_numbers] == 1, 1) 
N = len(idx_03457)

print("{:.2f}% of data contain a 0, 3, 4, 5, or 7".format(np.sum(idx_03457) / N * 100))

images_03457 = mnist.train.images[idx_03457]
labels_03457 = mnist.train.labels[:, five_numbers][idx_03457]

print('Shape of images with 0, 3, 4, 5, or 7: ', images_03457.shape)
print('Shape of labels with 0, 3, 4, 5, or 7: ', labels_03457.shape)

del mnist

# space_column = np.zeros(len(labels_03457)).reshape((-1, 1))
# labels_03457space = np.c_[labels_03457, space_column]

# print('Shape of Labels with space column: ', labels_03457space.shape)


In [9]:
key = '03457'
fig, axes = plt.subplots(1, 10, figsize=(10, 2))
for ax, image, label in zip(axes, images_03457, labels_03457):
    ax.set_axis_off()
    ax.set_title(key[np.argmax(label)])
    ax.imshow(image.reshape((28, 28)))
display()

In [10]:
print(images_03457.shape)
print(labels_03457.shape)

In [11]:
joined = np.hstack((images_03457, labels_03457))
print(joined.shape)
big_data = joined
iterations = 15

for i in range(iterations): 
  np.random.shuffle(joined)
  big_data = np.vstack((big_data, joined))

print(big_data.shape)

In [12]:
images_03457 = big_data[:, :784]
labels_03457 = big_data[:, 784:]
del big_data
print(images_03457.shape)
print(labels_03457.shape)

In [13]:
i = 0
N = labels_03457.shape[0] - 3
num_three_digit_img_features = 3 * images_03457.shape[1]
num_categories_3dig_spaces   = 3 * labels_03457.shape[1]


three_digits_images = np.zeros((N, num_three_digit_img_features))
three_digits_labels = np.zeros((N, num_categories_3dig_spaces ))
while i < N:
    num1 = images_03457[i].reshape((28, 28))
    num2 = images_03457[i+1].reshape((28, 28))
    num3 = images_03457[i+2].reshape((28, 28))
    num123 = np.hstack((num1, num2, num3))
    three_digits_images[i] = num123.ravel()
    
    label_1 = labels_03457[i] 
    label_2 = labels_03457[i + 1]
    label_3 = labels_03457[i + 2]
    three_digits_labels[i] = np.hstack((label_1, label_2, label_3)).ravel()
    i+=1
    
del images_03457
del labels_03457

In [14]:
three_digit_key = np.array(list('034570345703457'))
last_sample = three_digits_images[-1]
last_label = three_digits_labels[-1]

print(last_label)
print(''.join(three_digit_key[three_digits_labels[-1].astype(np.bool)]))

In [15]:
fig, ax = plt.subplots(figsize=(2,2))
ax.set_axis_off()
ax.imshow(last_sample.reshape((28, 84)));
display(fig)

In [16]:
def remove_digit(digit, three_digit):
    two_digit = np.copy(three_digit)
    two_digit.reshape(28, 84)[:, (digit * 28) : ((digit + 1) * 28)] = 0
    return two_digit.ravel()

def revise_label(digit, label):
    label_copy = np.copy(label)
    label_copy[digit * 5 : (digit + 1) * 5] = 0
    return label_copy
    

remove_key = {0:'first', 1:'second', 2:'third'}

fig, axes = plt.subplots(1, 3, figsize=(20, 3))
for digit, ax in zip(np.arange(3), axes):
    new_digits = remove_digit(digit, last_sample)    
    new_label = revise_label(digit, last_label)
    
    ax.imshow(new_digits.reshape(28, 84))
    ax.set_title("{} digit removed".format(remove_key[digit]))
    ax.set_xlabel('Label\n {}'.format(new_label))
display(fig)    

In [17]:
p = 0.5 # Designate 50% of samples having a digit or two removed

sample_size = int(N * p)

space_idx = np.random.RandomState(1234).choice(np.arange(N), size=sample_size, replace=False)


In [18]:
for i in space_idx:
    num_digits_to_remove = np.random.randint(1, 3)
    digit_places = [0, 1, 2]  
    j = 0
    while j < num_digits_to_remove:
        digit_to_remove = np.random.choice(digit_places)
        three_digits_images[i] = remove_digit(digit_to_remove, three_digits_images[i])
        three_digits_labels[i] = revise_label(digit_to_remove, three_digits_labels[i])
        digit_places.remove(digit_to_remove)
        j += 1     

In [19]:
fig, axes = plt.subplots(5, 5, figsize=(20, 10))
for i, ax in enumerate(axes.ravel()):
    ax.imshow(three_digits_images[i].reshape(28, 84))
    ax.set_title("{}".format(three_digits_labels[i]), fontsize=8)
    ax.set_axis_off()
display(fig)

In [20]:
print(three_digits_images.shape)
print(three_digits_labels.shape)

In [21]:
print(three_digits_images.shape[0] * 0.75)
print(three_digits_images.shape[0] * 0.25)

In [22]:
train_images = three_digits_images[:325000]
train_labels = three_digits_labels[:325000]

test_images = three_digits_images[325000:]
test_labels = three_digits_labels[325000:]

del three_digits_images
del three_digits_labels

In [23]:
### Your code implementation goes here.

# Create the model
x = tf.placeholder(tf.float32, [None, 2352])
W = tf.Variable(tf.zeros([2352, 15]))
b = tf.Variable(tf.zeros([15]))
y = tf.matmul(x, W) + b

# Define loss and optimizer
y_ = tf.placeholder(tf.float32, [None, 15])

# The raw formulation of cross-entropy,
#
#   tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(tf.nn.softmax(y)),
#                                 reduction_indices=[1]))
#
# can be numerically unstable.
#
# So here we use tf.nn.softmax_cross_entropy_with_logits on the raw
# outputs of 'y', and then average across the batch.
cross_entropy = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y))
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)

sess = tf.InteractiveSession()
tf.global_variables_initializer().run()
# Train
for i in range(5000):
  batch_xs = train_images[(i * 65):(i + 1) * 65]
  batch_ys = train_labels[(i * 65):(i + 1) * 65]
  sess.run(train_step, feed_dict={x: batch_xs, y_: batch_ys})
  if i%100 == 0:
    print('step ', i)
    
# Test trained model
correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

# print(sess.run(accuracy, feed_dict={x: test_images,
#                                     y_: test_labels}))

### Feel free to use as many code cells as needed.



In [24]:
ksflkfjlks

In [25]:
### Your code implementation goes here.

####################################################################################
#                             Helper Functions                                     #
####################################################################################
# Weight initialization

def weight_variable(shape):
  initial = tf.truncated_normal(shape, stddev=0.1)
  return tf.Variable(initial)

def bias_variable(shape):
  initial = tf.constant(0.1, shape=shape)
  return tf.Variable(initial)

# Convolution stride of 1 and no padding
def conv2d(x, W):
  return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

# 2X2 pooling
def max_pool_2x2(x):
  return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')


####################################################################################
#                             Model Building                                       #
####################################################################################

# Create the model

# First convulution layer - 1 layer 32 features
W_conv1 = weight_variable([5, 5, 1, 32])
b_conv1 = bias_variable([32])

x_image = tf.reshape(x, [-1,28,84,1])

h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

# Second convulution layer - 32 layers 64 features
W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

# Densely connected layer - 1 layer 1024 features
W_fc1 = weight_variable([7 * 21 * 64, 1024])
b_fc1 = bias_variable([1024])

h_pool2_flat = tf.reshape(h_pool2, [-1, 7*21*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

# Apply Dropout
keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

# Final Readout Layer
W_fc2 = weight_variable([1024, 15])
b_fc2 = bias_variable([15])

y_conv = tf.matmul(h_fc1_drop, W_fc2) + b_fc2

####################################################################################
#                                  Model Training                                  #
####################################################################################

cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(y_conv, y_))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
sess.run(tf.global_variables_initializer())
for i in range(5000):
  batch_xs = train_images[(i * 65):(i + 1) * 65]
  batch_ys = train_labels[(i * 65):(i + 1) * 65]
  if i%10 == 0:
    train_accuracy = accuracy.eval(feed_dict={
        x:batch_xs, y_: batch_ys, keep_prob: 1.0})
    print("step %d, training accuracy %g"%(i, train_accuracy))
  train_step.run(feed_dict={x: batch_xs, y_: batch_ys, keep_prob: 0.9})

# print("test accuracy %g"%accuracy.eval(feed_dict={
#     x: three_digits_images[:27081], y_: three_digits_labels[:27081], keep_prob: 1.0}))

### Feel free to use as many code cells as needed.

In [26]:
kasjlkdfldsj

In [27]:
 print("test accuracy %g"%accuracy.eval(feed_dict={
     x: train_images[:-1], y_: train_labels[:-1], keep_prob: 1.0}))

In [28]:
from six.moves.urllib.request import urlretrieve
import os
import sys

last_percent_reported = None

def download_progress_hook(count, blockSize, totalSize):
    """A hook to report the progress of a download. This is mostly intended for users with
    slow internet connections. Reports every 5% change in download progress.
    """
    global last_percent_reported
    percent = int(count * blockSize * 100 / totalSize)

    if last_percent_reported != percent:
        if percent % 5 == 0:
            sys.stdout.write("%s%%" % percent)
            sys.stdout.flush()
        else:
            sys.stdout.write(".")
            sys.stdout.flush()
      
        last_percent_reported = percent

    
url = "http://ufldl.stanford.edu/housenumbers/"

def maybe_download(filename, expected_bytes, force=False):
    """Download a file if not present, and make sure it's the right size."""
    if force or not os.path.exists(filename):
        print('Attempting to download:', filename) 
        filename, _ = urlretrieve(url + filename, filename, reporthook=download_progress_hook)
        print('\nDownload Complete!')
    statinfo = os.stat(filename)
    if statinfo.st_size == expected_bytes:
        print('Found and verified', filename)
    else:
        raise Exception(
          'Failed to verify ' + filename + '. Can you get to it with a browser?')
    return filename

train_filename = maybe_download('train.tar.gz', 404141560)
test_filename = maybe_download('test.tar.gz', 276555967)
extra_filename = maybe_download('extra.tar.gz', 1955489752)

In [29]:
import tarfile

def maybe_extract(filename, force=False):
    root = os.path.splitext(os.path.splitext(filename)[0])[0]  # remove .tar.gz
    if os.path.isdir(root) and not force:
        # You may override by setting force=True.
        print('%s already present - Skipping extraction of %s.' % (root, filename))
    else:
        print('Extracting data for %s. This may take a while. Please wait.' % root)
        tar = tarfile.open(filename)
        sys.stdout.flush()
        tar.extractall()
        tar.close()
    return None
  
maybe_extract(train_filename)
maybe_extract(test_filename)
maybe_extract(extra_filename)

### Question 1
_What approach did you take in coming up with a solution to this problem?_

**Answer:**

### Question 2
_What does your final architecture look like? (Type of model, layers, sizes, connectivity, etc.)_

**Answer:**

### Question 3
_How did you train your model? How did you generate your synthetic dataset?_ Include examples of images from the synthetic data you constructed.

**Answer:**

----
## Step 2: Train a Model on a Realistic Dataset
Once you have settled on a good architecture, you can train your model on real data. In particular, the [Street View House Numbers (SVHN)](http://ufldl.stanford.edu/housenumbers/) dataset is a good large-scale dataset collected from house numbers in Google Street View. Training on this more challenging dataset, where the digits are not neatly lined-up and have various skews, fonts and colors, likely means you have to do some hyperparameter exploration to perform well.

### Implementation
Use the code cell (or multiple code cells, if necessary) to implement the first step of your project. Once you have completed your implementation and are satisfied with the results, be sure to thoroughly answer the questions that follow.

In [38]:


### Your code implementation goes here.
### Feel free to use as many code cells as needed.



### Question 4
_Describe how you set up the training and testing data for your model. How does the model perform on a realistic dataset?_

**Answer:**

### Question 5
_What changes did you have to make, if any, to achieve "good" results? Were there any options you explored that made the results worse?_

**Answer:**

### Question 6
_What were your initial and final results with testing on a realistic dataset? Do you believe your model is doing a good enough job at classifying numbers correctly?_

**Answer:**

----
## Step 3: Test a Model on Newly-Captured Images

Take several pictures of numbers that you find around you (at least five), and run them through your classifier on your computer to produce example results. Alternatively (optionally), you can try using OpenCV / SimpleCV / Pygame to capture live images from a webcam and run those through your classifier.

### Implementation
Use the code cell (or multiple code cells, if necessary) to implement the first step of your project. Once you have completed your implementation and are satisfied with the results, be sure to thoroughly answer the questions that follow.

In [47]:


### Your code implementation goes here.
### Feel free to use as many code cells as needed.



### Question 7
_Choose five candidate images of numbers you took from around you and provide them in the report. Are there any particular qualities of the image(s) that might make classification difficult?_

**Answer:**

### Question 8
_Is your model able to perform equally well on captured pictures or a live camera stream when compared to testing on the realistic dataset?_

**Answer:**

### Optional: Question 9
_If necessary, provide documentation for how an interface was built for your model to load and classify newly-acquired images._

**Answer:** Leave blank if you did not complete this part.

----
### Step 4: Explore an Improvement for a Model

There are many things you can do once you have the basic classifier in place. One example would be to also localize where the numbers are on the image. The SVHN dataset provides bounding boxes that you can tune to train a localizer. Train a regression loss to the coordinates of the bounding box, and then test it.

### Implementation
Use the code cell (or multiple code cells, if necessary) to implement the first step of your project. Once you have completed your implementation and are satisfied with the results, be sure to thoroughly answer the questions that follow.

In [56]:


### Your code implementation goes here.
### Feel free to use as many code cells as needed.



### Question 10
_How well does your model localize numbers on the testing set from the realistic dataset? Do your classification results change at all with localization included?_

**Answer:**

### Question 11
_Test the localization function on the images you captured in **Step 3**. Does the model accurately calculate a bounding box for the numbers in the images you found? If you did not use a graphical interface, you may need to investigate the bounding boxes by hand._ Provide an example of the localization created on a captured image.

**Answer:**

----
## Optional Step 5: Build an Application or Program for a Model
Take your project one step further. If you're interested, look to build an Android application or even a more robust Python program that can interface with input images and display the classified numbers and even the bounding boxes. You can for example try to build an augmented reality app by overlaying your answer on the image like the [Word Lens](https://en.wikipedia.org/wiki/Word_Lens) app does.

Loading a TensorFlow model into a camera app on Android is demonstrated in the [TensorFlow Android demo app](https://github.com/tensorflow/tensorflow/tree/master/tensorflow/examples/android), which you can simply modify.

If you decide to explore this optional route, be sure to document your interface and implementation, along with significant results you find. You can see the additional rubric items that you could be evaluated on by [following this link](https://review.udacity.com/#!/rubrics/413/view).

### Optional Implementation
Use the code cell (or multiple code cells, if necessary) to implement the first step of your project. Once you have completed your implementation and are satisfied with the results, be sure to thoroughly answer the questions that follow.

In [63]:


### Your optional code implementation goes here.
### Feel free to use as many code cells as needed.



### Documentation
Provide additional documentation sufficient for detailing the implementation of the Android application or Python program for visualizing the classification of numbers in images. It should be clear how the program or application works. Demonstrations should be provided.

_Write your documentation here._

> **Note**: Once you have completed all of the code implementations and successfully answered each question above, you may finalize your work by exporting the iPython Notebook as an HTML document. You can do this by using the menu above and navigating to  
**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission.